In [1]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as sig

In [2]:
df = pd.read_csv("../../db/data/merge/tertiary/idxs_.csv",index_col=0)
df_logs = pd.read_csv("../../db/data/merge/tertiary/logs_.csv",index_col=0)
h = pd.read_csv("../../db/data/merge/tertiary/higs_.csv",index_col=0)
l = pd.read_csv("../../db/data/merge/tertiary/lows_.csv",index_col=0)


## *Bollinger Bands Function* 
#### (we are not going to use this function, just to "show off")

In [3]:
def get_bollinger_bands(df, rate=5, sigma_start=2, sigma_stop=2):
    
    sma = df.rolling(rate).mean()
    bollinger_bands = {"sma":sma}
    std = df.rolling(rate).std()
    for sigma in range(sigma_start, sigma_stop+1):
        up = sma + std * sigma
        down = sma - std * sigma
        bollinger_bands[f'up{sigma}'] = up
        bollinger_bands[f'down{sigma}'] = down
    bol = pd.DataFrame.from_dict(bollinger_bands)
    bol.set_index(df.index, inplace=True)
    return bol


## *Bollinger reduced*

In [4]:
def bollinger_small(df, rate=5, sigma_start=2, sigma_stop=2):
    
    data = pd.DataFrame([])
    
    for currency in df.columns:
        data[f'{currency}_sma'] = df[currency].rolling(rate).mean()
        data[f'{currency}_std']= df[currency].rolling(rate).std()
        
    data.index = df.index
    
    return data

## *Volatility Function*

In [5]:
def volatility(df,rate=240,window=506):
    
    data = pd.DataFrame([])
    for currency in df.columns:
        data[f"{currency}_vol1"] = df[currency].rolling(rate).std()/(window**0.5)
        data[f"{currency}_vol2"] = df[currency].rolling(rate).std()*(window**0.5)
    data.index = df.index
    
    return data

## *ATR Function*

In [6]:

# def atr(close: pd.DataFrame, low: pd.DataFrame, high: pd.DataFrame, window:int=14):
#     data = pd.DataFrame([])
#     ccy_pairs = close.columns
#     for pair in ccy_pairs:
#         data[f'close_{pair}'] = close[pair]
#         data[f'high_{pair}'] = high[pair]
#         data[f'low_{pair}'] = low[pair]
#         data[f'tr0_{pair}'] = data[f'high_{pair}'] - data[f'low_{pair}']
#         data[f'tr1_{pair}'] = np.abs(data[f'high_{pair}'] - data[f'close_{pair}'].shift())
#         data[f'tr2_{pair}'] = np.abs(data[f'low_{pair}'] - data[f'close_{pair}'].shift())
#         data[f'tr_{pair}'] = data[[f'tr0_{pair}', f'tr1_{pair}', f'tr2_{pair}']].max(axis=1)
#         data[f'atr_{pair}'] = ema(data[f'tr_{pair}'], window)
#     data.dropna(inplace=True)
#     return datab

In [7]:
def ema(df: pd.Series, window: int):
    return df.ewm(alpha=1/window, adjust=False).mean()
def atr(idxs: pd.DataFrame, low: pd.DataFrame, high: pd.DataFrame, window:int=14):
    data = pd.DataFrame([])
    for currency in idxs.columns:
        data[f'idxs_{currency}'] = idxs[currency]
        data[f'high_{currency}'] = high[currency]
        data[f'low_{currency}'] = low[currency]
        data[f'tr0_{currency}'] = data[f'high_{currency}'] - data[f'low_{currency}']
        data[f'tr1_{currency}'] = np.abs(data[f'high_{currency}'] - data[f'idxs_{currency}'].shift())
        data[f'tr2_{currency}'] = np.abs(data[f'low_{currency}'] - data[f'idxs_{currency}'].shift())
        data[f'tr_{currency}'] = data[[f'tr0_{currency}', f'tr1_{currency}', f'tr2_{currency}']].max(axis=1)
        data[f'atr_{currency}'] = ema(data[f'tr_{currency}'], window)
        data.drop(columns=[f'idxs_{currency}',f'tr0_{currency}',f'tr1_{currency}',f'tr2_{currency}',f'tr_{currency}'], inplace=True)
    return data

In [8]:
atr = atr(idxs=df,low=l,high=h)
atr

,high_AUD,low_AUD,atr_AUD,high_CAD,low_CAD,atr_CAD,high_CHF,low_CHF,atr_CHF,high_EUR,...,atr_GBP,high_JPY,low_JPY,atr_JPY,high_NZD,low_NZD,atr_NZD,high_USD,low_USD,atr_USD
2005-01-03 00:00:00+00:00,0.121767,0.069635,0.052132,0.093836,0.087292,0.006544,0.097443,0.098832,-0.001388,0.136736,...,0.035335,0.234702,0.064217,0.170485,0.105893,0.058206,0.047687,0.120767,0.079393,0.041374
2005-01-03 01:00:00+00:00,0.108162,0.071188,7.186180,0.108821,0.082313,7.143055,0.127666,0.101411,7.134324,0.084908,...,7.170483,0.202362,0.010323,7.300428,0.128195,0.111212,7.179194,0.102141,0.056625,7.177231
2005-01-03 02:00:00+00:00,0.151917,0.046523,13.811973,0.131936,0.063988,13.766291,0.081254,0.155121,13.753217,0.128043,...,13.794868,0.220438,0.007873,13.931883,0.156393,0.054079,13.811678,0.144058,0.059286,13.804790
2005-01-03 03:00:00+00:00,0.289592,0.200533,19.947426,0.202997,0.327007,19.915628,0.123332,0.290384,19.885015,0.268214,...,19.931312,0.121634,0.395422,20.094620,0.255152,0.297903,19.951254,0.350430,0.373030,19.949631
2005-01-03 04:00:00+00:00,0.168989,0.161007,25.632696,0.117269,0.104562,25.644476,0.200682,0.084745,25.570577,0.024570,...,25.636767,0.109637,0.187191,25.793609,0.167504,0.148885,25.674879,0.123726,0.109664,25.709806
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27 19:00:00+00:00,0.072443,0.086818,12.938105,0.038892,0.077746,117.885582,0.027977,0.138255,18.872995,0.022771,...,144.622670,0.044480,0.084266,29.608571,0.042426,0.084664,48.662301,0.061721,0.085835,305.274743
2022-12-27 20:00:00+00:00,0.072443,0.086818,12.924731,0.038892,0.077746,117.893071,0.027977,0.138255,18.857171,0.022771,...,144.652644,0.044480,0.084266,29.592281,0.042426,0.084664,48.644959,0.061721,0.085835,305.432314
2022-12-27 21:00:00+00:00,0.072443,0.086818,12.911370,0.038892,0.077746,117.900561,0.027977,0.138255,18.841360,0.022771,...,144.682624,0.044480,0.084266,29.576000,0.042426,0.084664,48.627624,0.061721,0.085835,305.589965
2022-12-27 22:00:00+00:00,0.072443,0.086818,12.898023,0.038892,0.077746,117.908051,0.027977,0.138255,18.825562,0.022771,...,144.712610,0.044480,0.084266,29.559727,0.042426,0.084664,48.610294,0.061721,0.085835,305.747698


## *Sharpe Ratio*

In [ ]:
# def sharpe():
#     Sharpe_Ratio = portf_val[‘Daily Return’].mean() / portf_val[‘Daily Return’].std()